In [2]:
import requests
import json
import pandas as pd
import numpy as np
import pymysql
import os

In [3]:
# 1. 영화id를 사용하기 위해 영화정보 파일가져오기 
tmdb_movie_list = pd.read_csv('../data-files/total_tmdbmovielist.csv', index_col=0) 

C:\Users\user\.conda\envs\ykseo\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,4,7,12,17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# 2. 영화정보 중 id만 가져오기
tmdb_movie_list.info()
tmdb_movie_id_list = tmdb_movie_list[["id"]]
# del tmdb_movie_list

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630159 entries, 0 to 630185
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   adult                  630158 non-null  object 
 1   backdrop_path          150102 non-null  object 
 2   belongs_to_collection  11995 non-null   object 
 3   budget                 630157 non-null  object 
 4   genres                 630155 non-null  object 
 5   homepage               69801 non-null   object 
 6   id                     630156 non-null  object 
 7   imdb_id                396414 non-null  object 
 8   original_language      630157 non-null  object 
 9   original_title         630130 non-null  object 
 10  overview               25238 non-null   object 
 11  popularity             630119 non-null  object 
 12  poster_path            426255 non-null  object 
 13  production_companies   630119 non-null  object 
 14  production_countries   630088 non-nu

In [5]:
print(tmdb_movie_id_list.shape)
tmdb_movie_id_list.head()

(630159, 1)


,id
0,2.0
1,3.0
2,5.0
3,6.0
4,8.0


In [9]:
## 영화id를 이용하여 crew정보 crawling  (너무 오래걸림)

all_crew_list = []
all_cast_list = []
error_list = []
for idx,  movie_id in enumerate(tmdb_movie_id_list["id"].values):
    try :
        crew_url="https://api.themoviedb.org/3/movie/{0}?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
        response = requests.get(crew_url.format(movie_id))
        json_object = response.content
        movie_data = json.loads(json_object)
        
        # crew_list += movie_data["credit"]['crew']
        crew_list = movie_data["credits"]["crew"]
        for crew in crew_list:
            crew["movie_id"] = movie_id
            all_crew_list.append(crew)
    except:
        error_list.append(movie_id)

    if idx == 500:
        break

In [10]:
tmdb_movie_id_list["id"].values

array([2.0, 3.0, 5.0, ..., 879997.0, 879998.0, 879999.0], dtype=object)

In [11]:
# crew 정보 csv파일로 저장 
crews=pd.DataFrame(all_crew_list)
crews
# crews.to_csv("crews.csv", mode='w')

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department,job,movie_id
0,False,2,16767,Directing,Aki Kaurismäki,Aki Kaurismäki,1.400,/t0pdsYfLts7p01OBN1P9UHp1Z0l.jpg,52fe420dc3a36847f800001f,Writing,Screenplay,2.0
1,False,2,16767,Directing,Aki Kaurismäki,Aki Kaurismäki,1.400,/t0pdsYfLts7p01OBN1P9UHp1Z0l.jpg,52fe420dc3a36847f8000025,Directing,Director,2.0
2,False,2,16767,Directing,Aki Kaurismäki,Aki Kaurismäki,1.400,/t0pdsYfLts7p01OBN1P9UHp1Z0l.jpg,5be040dac3a3682b34043d42,Production,Producer,2.0
3,False,2,16769,Camera,Timo Salminen,Timo Salminen,1.400,/a8rH6eJ0XY1zQph18eNDj7MC8y4.jpg,52fe420dc3a36847f800003b,Camera,Director of Photography,2.0
4,False,1,54766,Editing,Raija Talvio,Raija Talvio,0.600,None,52fe420dc3a36847f8000041,Editing,Editor,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
32687,False,1,9028,Production,Sandrine Ageorges,Sandrine Ageorges,0.600,None,52fe4260c3a36847f80199f5,Production,Unit Production Manager,628.0
32688,False,2,17016,Directing,Neil Jordan,Neil Jordan,1.960,/uZ7CvCLTKb9qxjU4VsTNeyMRucd.jpg,52fe4260c3a36847f8019a1b,Directing,Director,628.0
32689,False,2,928502,Sound,Matthias Gohl,Matthias Gohl,0.600,/3LcqKXTCgKKplje0zg5E1BDjOaj.jpg,5ba763fb9251412ef10231e9,Sound,Music Score Producer,628.0
32690,False,2,1900639,Sound,Stephen McLaughlin,Stephen McLaughlin,0.996,/q7WO4q0OoJPmyN15NJAO8fh5q0p.jpg,5ba643ff9251412f0400adbc,Sound,Scoring Mixer,628.0


In [ ]:
# crew 정보 DB(mysql)에 저장 (테이블 만들어야 함)

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

with conn.cursor() as cursor: # with가 종료될 때 cursor.close() 자동 호출
    cursor.execute()
    cursor.execute()
    conn.commit() # 이전에 실행된 SQL 결과를 확정

conn.close()

In [32]:
# cast 정보 csv파일로 저장 
casts=pd.DataFrame(all_cast_list)
casts.to_csv("crews.csv", mode='w')

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id
0,False,2,54768,Acting,Turo Pajala,Turo Pajala,1.725,/b6JdzqTn6UFYf4DouHbbE8Ypk4r.jpg,3,Taisto Olavi Kasurinen,52fe420dc3a36847f8000029,0,2.0
1,False,1,54769,Acting,Susanna Haavisto,Susanna Haavisto,1.289,/lhjoqlBFCvshchMAd2lQLWV7z7I.jpg,4,Irmeli Katariina Pihlaja,52fe420dc3a36847f800002d,1,2.0
2,False,2,4826,Acting,Matti Pellonpää,Matti Pellonpää,1.607,/1Qzhkkp3wFE2NMGKVyOra9931q2.jpg,5,Mikkonen,52fe420dc3a36847f8000031,2,2.0
3,False,2,54770,Acting,Eetu Hilkamo,Eetu Hilkamo,0.600,None,6,Riku,52fe420dc3a36847f8000035,3,2.0
4,False,2,1177496,Acting,Erkki Pajala,Erkki Pajala,0.766,/lJ7GmiHT1UPxmURqQCLT4L5PIMa.jpg,12,Miner,5e1cda97d5dbc20017f439bb,4,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,False,1,105988,Acting,Galyn Görg,Galyn Görg,2.451,/sr3PUWk3Qo0b6cYh3pdSloBAgbx.jpg,20,Clarissa,52fe420dc3a36847f8000289,12,6.0
84,False,1,22621,Acting,Angela Alvarado,Angela Alvarado,1.443,None,21,Rita,52fe420dc3a36847f800028d,13,6.0
85,False,0,1226696,Acting,Will Zahrn,Will Zahrn,0.600,/5JDnuufySepQ1fig7Hga4xL4nio.jpg,34,Charley,560c6b95c3a368681f0064e8,14,6.0
86,False,1,901,Acting,Rita Lengyel,Rita Lengyel,1.048,None,7,Alexandra,52fe420dc3a36847f8000333,0,9.0
